In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal, Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
import operator

In [2]:
generator_llm = ChatOpenAI(model = 'gpt-4o-mini')
evaluator_llm = ChatOpenAI(model = 'gpt-4o-mini')
optimizer_llm = ChatOpenAI(model = 'gpt-4o-mini')

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [3]:
from pydantic import BaseModel, Field
class TweetEvaluation(BaseModel):
    evaluation : Literal['approved', 'needs_improvement'] = Field(..., description = 'Final evaluation result.')
    feedback:str = Field(..., description='feedback for the tweet.')

In [4]:
structured_evaluator_llm = evaluator_llm.with_structured_output(TweetEvaluation)


NameError: name 'evaluator_llm' is not defined

In [5]:
class TweetState(TypedDict):
    topic:str
    tweet:str
    evaluation: Literal['approved', 'needs_improvement']
    feedback:str
    iteration:int
    max_iteration:int

    tweet_history:Annotated[list[str], operator.add]
    feedback_history:Annotated[list[str], operator.add]
    

In [6]:
def generate_tweet(state:TweetState):
    messages = [SystemMessage(content='you are a funny and clever Twitter/X influencer.'),
                HumanMessage(content=f"""Write a short, original and hilarious tweet on the topic: "{state['topic']}".
                             
                             Rules:
                                - Do NOT use question-answer format.
                                - Max 300 characters.
                                - use observational humor, irony, sarcasm, or cultural references.
                                - Think in meme logic, punchlines, or relatable takes.
                                - Use simple, day to day english
                             """
                             )]
    response = generator_llm.invoke(messages).content
    return {'tweet':response, 'tweet_history': [response]}

In [7]:
def evaluate_tweet(state:TweetState):
    messages = [
        SystemMessage(content='You are a ruthless, no-laugh-given Twitter critic. You evaluate twwets based on humor, originality,' \
        'virality and tweet format.'),
        HumanMessage(content=f"""
                     Evaluate the following tweet:
                     Tweet:"{state['tweet']}"

                     Use the criteria below to evaluate the tweet:

                     1. Orignality - Is this fresh, or have you seen it a hundred times before?
                     2. Humor - Did it genuinely mak eyou smile, laugh, or chuckle?
                     3. Punchiness - Is it short, sharp, and scroll-stopping?
                     4. Virality Potential - Would people retweet or share it?
                     5. Format - Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and uinder 300 characters)?

                     Auto-reject if:
                     - It's written in question-answer format (e.g., 'Why did...' or 'What happens when...")
                     - It exceeds 300 characters
                     - It reads like a traditional setup-punchline joke
                     - Don't end with generic, throwaway, or deflating lines that weaken the humor (e.g., "Masterpieces 
                     of the auntie-uncle universe' or vague summaries)
                     ### Respond ONLY in structured format:
                     - evaluation: 'approved' or 'needs_improvement'Annotated
                     - feedback: One paragraph explaining the strengths and weaknesses
                    """
                     )
    ]
    response = structured_evaluator_llm.invoke(messages)
    return {'evalution':response.evaluation, 'feedback': response.feedback, 'feedback_history': [response.feedback]}

In [8]:
def optimize_tweet(state:TweetState):
    messages = [
        SystemMessage(content='You punch up tweets for virality and humor based on given feedback.'),
        HumanMessage(content=f"""additional_kwargs=
                     Improve the tweet based on this feedback:
                     "{state['feedback']}"
                     Topic: "{state['topic']}"
                     Original Tweet: {state['tweet']}
                     Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 300 characters.
                     """)
    ]
    response = optimizer_llm.invoke(messages).content
    iteration = state['iteration'] + 1
    return {'tweet':response, 'iteration':iteration, 'tweet_history': [response]}

In [9]:
def route_evaluation(state:TweetState):
    if state['evaluation'] == 'approved' or state['iteration'] >= state['max_iteration']:
        return 'approved'
    else:
        return 'needs_improvement'

In [ ]:
graph = StateGraph(TweetState)
graph.add_node('generate', generate_tweet)
graph.add_node('evaluate', evaluate_tweet)
graph.add_node('optimize', optimize_tweet)

graph.add_edge(START, 'generate')
graph.add_edge('generate', 'evaluate')
graph.add_conditional_edges('evaluate', route_evaluation, {'approved':END, 'needs_improvement':'optimize'})
graph.add_edge('optimize', 'evaluate')

workflow = graph.compile()               

In [ ]:
intial_state = {
    "topic":'GenAI',
    'iteration': 1,
    'max_iteration': 5
}
result = workflow.invoke(intial_state)

In [ ]:
from IPython.display import Image
Image(workflow.get_graph().draw_mermaid_png())

In [ ]:
for tweet in result['tweet_history']:
    print(tweet)